In [7]:
# Read data from file
import numpy as np
def read(filename):
    with open(filename, 'r') as f:
        N, K = map(int, f.readline().split())
        d = [0] + list(map(float, f.readline().split()))
        t = np.zeros((N + 1, N + 1))
        for i in range(N + 1):
            t[i] = list(map(float, f.readline().split()))

    return N, K, d, t

def creat_cost(N, d, t):
    cost_matrix = np.zeros((N + 1, N + 1), dtype = int)
    for i in range(N + 1):
        for j in range(N + 1):
            if i != j:
                cost_matrix[i][j] = d[j] + t[i][j]

    return cost_matrix

def print_result(K, N, x, objective, time_pro, filename):
    with open(filename, 'w') as f:
        f.write("Objective: " + str(objective) + '\n')
        f.write("Time processing: " + str(time_pro) + '\n')
        f.write("Solution: \n")
        f.write(str(K) + '\n')
        for k in range(K):
            f.write(str(sum(x[i][j][k] for j in range(N + 1) for i in range(N + 1))) + '\n')
            current_pos = 0
            f.write(str(0) + ' ')

            while True:
                for j in range(N + 1):
                    if x[current_pos][j][k] == 1:
                        f.write(str(j) + ' ')
                        current_pos = j
                        break
                if current_pos == 0:
                    break

            f.write('\n')

def print_no_solution(filename):
    with open(filename, 'w') as f:
        f.write("No solution")


In [8]:
from ortools.sat.python import cp_model

def CP_solve_maintenance_problem(file_data):
    data_path = "Data\\" + file_data
    N, K, d, t = read(data_path)
    cost_matrix = creat_cost(N, d, t)

    model = cp_model.CpModel()

    # Biến quyết định x[i][j][k] = 1 Nếu nhân viên k đi từ i đến j
    x = [[[model.NewIntVar(0, 1, f'x_{i}_{j}_{k}') for k in range(K)] for j in range(N + 1)] for i in range(N + 1)]
    # Biến y[k] = 1: Nhân viên k hoạt động
    y = [model.NewIntVar(0, 1, 'y[%i]' % k) for k in range(K)]
    # Biến p[i][k]: Thứ tự khách hàng i được phục vụ xong bởi nhân viên k
    p = [[model.NewIntVar(0, N, 'p[%i,%i]' % (i, k)) for k in range(K)] for i in range(N + 1)]

    # Ràng buộc: Mỗi khách hàng chỉ được phục vụ bởi 1 nhân viên
    for j in range(1, N + 1):
        model.Add(sum(x[i][j][k] for i in range(N + 1) for k in range(K)) == 1)

    # Ràng buộc: Mỗi nhân viên chỉ di chuyển nhiều nhất y[k] chuyến
    for k in range(K):
        condition1 = model.NewBoolVar('condition1')
        model.Add(y[k] == 0).OnlyEnforceIf(condition1)
        model.Add(sum(x[0][j][k] for j in range(1, N + 1)) == 0).OnlyEnforceIf(condition1)

    # Ràng buộc: Mỗi nhân viên bắt đầu từ 0 và kết thúc tại 0
    for k in range(K):
        model.Add(sum(x[0][j][k] for j in range(1, N + 1)) == y[k])
        model.Add(sum(x[i][0][k] for i in range(1, N + 1)) == y[k])
    
    for k in range(K):    
        for i in range(N + 1):
            model.Add(x[i][i][k] == 0)
    
    # Ràng buộc: Đến và rời khách hàng i bởi 1 nhân viên
    for j in range(0, N + 1):
        for k in range(K):
            model.Add(sum(x[i][j][k] for i in range(N + 1)) == sum(x[j][i][k] for i in range(N + 1)))


   # Ràng buộc: Nếu x[i][j][k] = 1 thì p[j][k] >= p[i][k] + 1
    for k in range(K):
        for i in range(N + 1):
            for j in range(1, N + 1):
                if i != j:
                    condition2 = model.NewBoolVar('condition2')
                    model.Add(x[i][j][k] == 0).OnlyEnforceIf(condition2.Not())
                    model.Add(x[i][j][k] == 1).OnlyEnforceIf(condition2)
                    model.Add(p[j][k] >= p[i][k] + 1).OnlyEnforceIf(condition2)

    max_work_time = model.NewIntVar(0, 1000000, 'max_work_time')
    for k in range(K):
        model.Add(max_work_time >= (sum(x[i][j][k] * cost_matrix[i][j] for i in range(N + 1) for j in range(N + 1))))

    model.Minimize(max_work_time)

    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    result_path = "Result_CP\\" + file_data
    if status == cp_model.OPTIMAL:
        objective = solver.ObjectiveValue()
        time_pro = solver.WallTime()
        x = [[[solver.Value(x[i][j][k]) for k in range(K)] for j in range(N + 1)] for i in range(N + 1)]
        print_result(K, N, x, objective, time_pro, result_path)
        

    else:
        print_no_solution(result_path)

In [9]:
data = ["data_5_2.txt", "data_5_3.txt", "data_10_2.txt", "data_10_3.txt", "data_10_5.txt", "data_15_3.txt", "data_15_5.txt", "data_15_7.txt", "data_20_5.txt", "data_20_7.txt", "data_20_10.txt", "data_50_5.txt", "data_50_10.txt", "data_50_20.txt"]

In [10]:
for data_file in data:
    CP_solve_maintenance_problem(data_file)